In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [3]:
import os

In [14]:
data_dir = "../ml-25m/"
# movies
movies_df = pd.read_csv(data_dir+"movies.csv")
links_df = pd.read_csv(data_dir+"links.csv")
scores_df = pd.read_csv(data_dir+"genome-scores.csv")
tag_df = pd.read_csv(data_dir+"genome-tags.csv")

# user
ratings_df = pd.read_csv(data_dir+ "ratings.csv")
tags_df = pd.read_csv(data_dir+"tags.csv")

In [8]:
movies_df[:3]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [9]:
ratings_df[:3]

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [11]:
links_df[:3]

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [12]:
tags_df[:3]

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598


In [15]:
scores_df[:3]

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250


In [16]:
tag_df[:3]

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century


In [36]:
tmp1 = ratings_df.groupby("userId").count()['rating'].min()
tmp2 = ratings_df.groupby("movieId").count()['rating'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [41]:
tmp1 = sum(ratings_df.groupby("movieId").count()['rating'] == 1)
tmp2 = ratings_df['movieId'].drop_duplicates().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

10298 out of 59047 movies are rated by only one user


## show the number of user

In [50]:
# user_number
user_count = ratings_df['userId'].drop_duplicates().count()
user_count

162541

## show the number of movie

In [51]:
# movie_number
movie_count = movies_df['movieId'].drop_duplicates().count()
movie_count

62423

## show the number of movie rated

In [54]:
movie_rated = movies_df['movieId'].isin(ratings_df).count()
movie_rated

62423

# check if there are missing values in rating as well here

In [59]:
movie_missing_value = ratings_df[ratings_df.isnull().T.any()]
movie_missing_value

,userId,movieId,rating,timestamp


# movie genres

In [66]:
movie_genres = movies_df['genres']
movie_genres[:10]

0    Adventure|Animation|Children|Comedy|Fantasy
1                     Adventure|Children|Fantasy
2                                 Comedy|Romance
3                           Comedy|Drama|Romance
4                                         Comedy
5                          Action|Crime|Thriller
6                                 Comedy|Romance
7                             Adventure|Children
8                                         Action
9                      Action|Adventure|Thriller
Name: genres, dtype: object

In [74]:
movie_genres=movies_df.drop('genres', axis=1).join(movies_df['genres'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).rename('splited_genres'))
movie_genres[:10]

,movieId,title,splited_genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance


In [80]:
category_movie_count  = movie_genres.splited_genres.value_counts()  
print("Number of categories:",len(category_movie_count))
category_movie_count[:10]

Number of categories: 20


Drama                 25606
Comedy                16870
Thriller               8654
Romance                7719
Action                 7348
Horror                 5989
Documentary            5605
Crime                  5319
(no genres listed)     5062
Adventure              4145
Name: splited_genres, dtype: int64

##  The check of the data has been finished,
##  next I input the data into Deep and Wide